<a href="https://colab.research.google.com/github/ajrotert/Machine-Learning/blob/master/BookGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk

In [0]:
with open('/content/drive/My Drive/Data/HungerGames.txt') as stream:
  text = stream.read()

chars = sorted(list(set(text)))
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i, c in enumerate(chars))

In [3]:
print(text[:500])

﻿The Hunger Games


The Hunger Games 1by Suzanne Collins

PART I"THE TRIBUTES"

1.

When I wake up, the other side of the bed is cold. My fingers stretch out, seeking Prims warmth but finding only the rough canvas cover of the mattress. She must have had bad dreams and climbed in with our mother. Of course, she did. This is the day of the reaping. I prop myself up on one elbow. Theres enough light in the bedroom to see them. My little sister, Prim, curled up on her side, cocooned in my mothers b


In [5]:
maxlen = 40
step = 3
sentences = []
next_char = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i+maxlen])
  next_char.append(text[i+maxlen])
print(len(sentences))

173964


In [6]:
print(sentences[0:20])

['\ufeffThe Hunger Games\n\n\nThe Hunger Games 1by', 'e Hunger Games\n\n\nThe Hunger Games 1by Su', 'unger Games\n\n\nThe Hunger Games 1by Suzan', 'er Games\n\n\nThe Hunger Games 1by Suzanne ', 'Games\n\n\nThe Hunger Games 1by Suzanne Col', 'es\n\n\nThe Hunger Games 1by Suzanne Collin', '\n\nThe Hunger Games 1by Suzanne Collins\n\n', 'he Hunger Games 1by Suzanne Collins\n\nPAR', 'Hunger Games 1by Suzanne Collins\n\nPART I', 'ger Games 1by Suzanne Collins\n\nPART I"TH', ' Games 1by Suzanne Collins\n\nPART I"THE T', 'mes 1by Suzanne Collins\n\nPART I"THE TRIB', ' 1by Suzanne Collins\n\nPART I"THE TRIBUTE', 'y Suzanne Collins\n\nPART I"THE TRIBUTES"\n', 'uzanne Collins\n\nPART I"THE TRIBUTES"\n\n1.', 'nne Collins\n\nPART I"THE TRIBUTES"\n\n1.\n\nW', ' Collins\n\nPART I"THE TRIBUTES"\n\n1.\n\nWhen', 'llins\n\nPART I"THE TRIBUTES"\n\n1.\n\nWhen I ', 'ns\n\nPART I"THE TRIBUTES"\n\n1.\n\nWhen I wak', '\nPART I"THE TRIBUTES"\n\n1.\n\nWhen I wake u']


In [0]:
import numpy as np

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i,t,char_indices[char]] = 1
  y[i,char_indices[next_char[i]]] = 1

In [0]:
%tensorflow_version 2.x

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [EarlyStopping(monitor="loss", patience=2), ModelCheckpoint(filepath="bestmodel.h5", monitor="loss", save_best_only=True)]

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               104448    
_________________________________________________________________
dense (Dense)                (None, 75)                9675      
_________________________________________________________________
activation (Activation)      (None, 75)                0         
Total params: 114,123
Trainable params: 114,123
Non-trainable params: 0
_________________________________________________________________


In [14]:
epochs = 30
batch_size = 128
model.fit(x, y, batch_size=batch_size, epochs=epochs, callbacks=callbacks)
model.save_weights('/content/drive/My Drive/Data/lstm_weights.h5')

Epoch 1/30
1360/1360 [==============================] - 160s 117ms/step - loss: 1.9465
Epoch 2/30
1360/1360 [==============================] - 161s 118ms/step - loss: 1.5978
Epoch 3/30
1360/1360 [==============================] - 161s 118ms/step - loss: 1.5018
Epoch 4/30
1360/1360 [==============================] - 161s 118ms/step - loss: 1.4497
Epoch 5/30
1360/1360 [==============================] - 161s 118ms/step - loss: 1.4169
Epoch 6/30
1360/1360 [==============================] - 161s 118ms/step - loss: 1.3918
Epoch 7/30
1360/1360 [==============================] - 161s 118ms/step - loss: 1.3732
Epoch 8/30
1360/1360 [==============================] - 162s 119ms/step - loss: 1.3562
Epoch 9/30
1360/1360 [==============================] - 161s 119ms/step - loss: 1.3432
Epoch 10/30
1360/1360 [==============================] - 162s 119ms/step - loss: 1.3317
Epoch 11/30
1360/1360 [==============================] - 163s 120ms/step - loss: 1.3217
Epoch 12/30
1360/1360 [==================

In [0]:
import random

def sample(preds, temp=1.0):
  preds= np.asarray(preds).astype('float64')
  preds=np.log(preds)/temp
  exp_preds = np.exp(preds)
  preds= exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [17]:
import sys
start_index = random.randint(0,len(text) - maxlen -1)
for diversity in [.05, .2, .5, 1.0]:
  print()
  print('----- diversity:', diversity)
  generated = ''
  sentence = text[start_index: start_index+maxlen]
  generated += sentence
  print('----- Generting with seed: ', sentence)
  sys.stdout.write(generated)
  for i in range(400):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x[0,t,char_indices[char]] = 1.0
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    generated += next_char
    sentence = sentence[1:] + next_char
    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()


----- diversity: 0.05
----- Generting with seed:  you make it back, will you?

Count on it
you make it back, will you?

Count on it and the start of the blood and the start of the blood and the start of the blood and the start of the blood and the start of the blood and the start of the blood and the start of the blood and the start of the blood and the start of the blood and the start of the blood and the start of the stage of the blood. I see the blood. The smell of the blood. I see the blood. I see the blood. I see the blo

----- diversity: 0.2
----- Generting with seed:  you make it back, will you?

Count on it
you make it back, will you?

Count on it and the worst the stage of the blood. The sky on the woods and the blood and the ground the woods and the start of blood. I want to the boy from District 11

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


. The course of the stream and the boy from District 11 I see the back of the blood. I see the woods are the stage and the reaping of blood. The boy from District 11. The slidies to the door. I see the stage and the blood and so the blood and 

----- diversity: 0.5
----- Generting with seed:  you make it back, will you?

Count on it
you make it back, will you?

Count on it. As I start the blood. I see its at a station. She should while I cant see the woods were not that I can feel the peecial and when I reach to the last of call my strange who catch the love pressimary. I so the other does. I reach the sense he says with the sense the breal. I see I cant start the woods take still screaminated the looker and this doesnt really another a botteam with the breakfast a

----- diversity: 1.0
----- Generting with seed:  you make it back, will you?

Count on it
you make it back, will you?

Count on its coal clear. I dont put the conversar comelifion. In the screen of intouray minect, and we c